In [ ]:
from src.segment import Segmenter
from src.synthetic_data import Point, Points, DistanceTransform 
from src.VMSI import *

## Example with Synthetic Data

### Generate the data 

In [ ]:
config = {
          'max_x' : 256,
          'max_y' : 256,
          'min_p' : 0.001,
          'max_p' : 0.004,
          'num_points' : 50
         }

generating_points = Points(**config)

dtr = DistanceTransform(generating_points)
dtr.compute_transform()
dtr.visualize_transform()

image = dtr.transform.copy()

### Segment it

In [ ]:
seg = Segmenter()
seg.segment(image.copy(), diameter=None)
seg.visualize('outlines', overlay=True)

### Infer the tensions

In [ ]:
# create the model 
model = VMSI(cell_pairs = seg.pairs(), edges = seg.edges(), num_cells = len(seg.cells[0]), 
             cells = seg.cells[0], barrycenters = seg.barrycenters[0], 
             edge_cells = seg.get_edge_cells(), height=256, width=256)

# fit the model parameters
q, z, p = model.fit()

# get the CAP tiling
CAP_tiling = model.CAP(image.copy(), q, z, p)

# view the CAP tiling
plt.figure(figsize=(15, 10))
plt.imshow(CAP_tiling)
plt.show()

# evaluate the model
evaluate(model, seg, dtr)

## Example with Real Data

### Load the Data

In [ ]:
image = cv2.resize(cv2.imread('Images/test.png.jpg', 0), (256, 256))

### Segment it

In [ ]:
seg = Segmenter() 
seg.segment(image, diameter=None) 
seg_img = seg.visualize('outlines', overlay=True, return_img=True)

### Infer the tensions

In [ ]:
# create the model 
model = VMSI(cell_pairs = seg.pairs(), edges = seg.edges(), num_cells = len(seg.cells[0]), 
             cells = seg.cells[0], barrycenters = seg.barrycenters[0], 
             edge_cells = seg.get_edge_cells(), height=256, width=256)

# fit the model parameters
q, z, p = model.fit()

# get the tiling
CAP_tiling model.CAP(np.float32(image.copy()), q, z, p)

# view the tension tiling
plt.figure(figsize=(15, 10))
plt.imshow(CAP_tiling)
plt.show()